In [1]:
from config import groq_cloud_key
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from groq import Groq

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
client = Groq(
    api_key=groq_cloud_key,
)

In [4]:
es_client = Elasticsearch('http://localhost:9200') 


In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            #"section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [7]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [8]:
query_homework="How do I execute a command in a running docker container?"

In [9]:
def elastic_search_homework(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [28]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"""Q: {doc['question']}\nA: {doc['text']}\n\n""".strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [11]:
def llm(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192")
    
    return response.choices[0].message.content

In [29]:
query = 'How do I execute a command in a running docker container?'

def rag(query):
    search_results = elastic_search_homework(query)
    prompt = build_prompt(query, search_results)
    print(len(prompt))
    print(len((encoding.encode(prompt))))
    answer = llm(prompt)
    return answer

In [30]:
import tiktoken
encoding = tiktoken.encoding_for_model('gpt-4o')

In [31]:
rag(query)


2706
619


'To execute a command in a running Docker container, you can use the following command:\n\ndocker exec -it <container-id> bash\n\nReplace `<container-id>` with the actual ID of your running Docker container. You can find the container ID by running the command `docker ps`.'

In [16]:
search_results=elastic_search_homework(query)

In [17]:
prompt=build_prompt(query, search_results)

In [18]:
len(encoding.encode(prompt))

629